<a href="https://colab.research.google.com/github/sk3271/Data-Science-Projects-/blob/main/News_Prediction_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # FAKE NEWs PREDICTION MODEL

##Dataset Description
train.csv: A full training dataset with the following attributes:

id: unique id for a news article

title: the title of a news article

author: author of the news articl

text: the text of the article; could be incomplete

label: a label that marks the article as potentially unreliable

1: unreliable
0: reliable

<b> Important Library </b>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 
import re # for searching text in document
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

Loading dataset

In [ ]:
!kaggle competitions download -c fake-news

fake-news.zip: Skipping, found more recently modified local copy (use --force to force download)


Unzip training File

In [ ]:
!unzip fake-news.zip

Archive:  fake-news.zip
replace submit.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


Data Pre-processing

In [ ]:
df=pd.read_csv('/content/train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
df.shape

(20800, 5)

Checking for null values

In [ ]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

Replacing null values with empty string 

In [ ]:
df=df.fillna('')

In [ ]:
df['content']=df['author']+' '+df['title']

In [ ]:
print(df['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


separating the data and lable

In [ ]:
X=df.drop('label',axis=1)

In [ ]:
Y=df.label

In [ ]:
print(X)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

####Steming:
 It is the process of reducing a word from root word.

 example:- actor, actress, acting  --> act

In [ ]:
port_stem= PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub("[^a-zA-Z]",' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content= ' '.join(stemmed_content)
  return stemmed_content

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
print(STOPWORDS)

{'his', "shan't", 'once', 'or', 'each', "mightn't", "shouldn't", "aren't", 'my', "needn't", "should've", "didn't", 't', 'ours', "wasn't", 'won', 'theirs', 'only', 'yours', 'between', 'here', 'so', 'there', 'too', 'haven', "you'll", 'as', 'our', 'than', 'about', 'over', 'when', 'its', 'before', 'does', 'after', 'whom', 'why', 'needn', 'if', 'until', 'how', 'no', 'during', 'is', 'yourself', 'nor', "wouldn't", "doesn't", 'that', 'at', 'their', 'ourselves', 'in', 'them', 'then', 'some', 'under', 'into', 'o', 'through', 'd', 'she', 'what', 'for', 'of', 'having', "isn't", 'weren', 'don', 'again', 'such', 'few', 'did', 'are', 'be', 'you', 'very', 'those', 'on', 'the', 'has', 'this', 'hasn', 'but', "weren't", "you've", 'am', 'her', 'mightn', 'yourselves', 'an', 've', "mustn't", 'been', "don't", "won't", 'mustn', 'himself', 'ain', "you're", 'with', 'where', 'had', 'off', 'couldn', 'wasn', 'to', 'not', 'were', 'above', 'me', "hasn't", 'both', 'being', 'i', 'most', 'ma', 'doing', 'aren', 'he', 'm

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['content']=df['content'].apply(stemming)

In [ ]:
print(df.content)

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [ ]:
X=df['content'].values
Y=df['label'].values

In [ ]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']


Converting textual data to numerical data

In [ ]:
vectortizer = TfidfVectorizer()
vectortizer.fit(X)
X=vectortizer.transform(X)

In [ ]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2)

Training the Model

In [ ]:
model=LogisticRegression()

In [ ]:
model.fit(X_train,Y_train)

LogisticRegression()

In [ ]:
y_pred=model.predict(X_train)
a_score=accuracy_score(y_pred,Y_train)

In [ ]:
a_score

0.9865985576923076

In [ ]:
x_pred=model.predict(X_test)
test_score=accuracy_score(x_pred,Y_test)

In [ ]:
test_score

0.9790865384615385

Making a Predictive System 

In [ ]:
X_news = X_test[0]

prediction = model.predict(X_news)
print(prediction)

if (prediction[0]==0):
  print("THE NEWS IS REAL")
else:
  print("THE NEWS IS FAKE")

[1]
THE NEWS IS FAKE
